In [1]:
import polars as pl

In [3]:
var_df = pl.read_excel('variables.xlsx')
var_df = (var_df
 .with_columns(
     pl.when(pl.col('fieldName').is_null()).then(pl.lit('true')).otherwise(pl.lit('false')).alias('disabled')
 )
 .with_columns(pl.col('fieldName').fill_null('null'))
 .with_columns(pl.col('yearsAvailable').fill_null('[]'))
 .with_row_index(name='variableId',offset=1)
 .select(pl.col('variableId','name','fieldName','valueType','yearsAvailable','disabled','category'))
 
)
var_df

variableId,name,fieldName,valueType,yearsAvailable,disabled,category
u32,str,str,str,str,str,str
1,"""Population - Older Adult""","""percentover65""","""percentage""","""[2018,2019,2020,2021,2022,2023…","""false""","""Character"""
2,"""Population - Youth""","""percentunder18""","""percentage""","""[2018,2019,2020,2021,2022,2023…","""false""","""Character"""
3,"""Age or Residents""","""null""","""numeric""","""[]""","""true""","""Character"""
4,"""Area""","""null""","""numeric""","""[]""","""true""","""Character"""
5,"""Population Density""","""null""","""numeric""","""[]""","""true""","""Character"""
…,…,…,…,…,…,…
84,"""Long Commute""","""null""","""percentage""","""[]""","""true""","""Transportation"""
85,"""Proximity to Public Transporta…","""null""","""percentage""","""[]""","""true""","""Transportation"""
86,"""Sidewalk Availablity""","""null""","""percentage""","""[]""","""true""","""Transportation"""


In [4]:
v_dicts = var_df.select(pl.col('variableId','name','fieldName','valueType','yearsAvailable','disabled')).to_dicts()

In [ ]:
with open('reg_exp_variables.json','w') as f:
    for var in v_dicts:
        print(var)
        f.write('{\n')
        for k,v in var.items():
            rec =  f"{k}: '{v}'" \
                if k in ['valueType','name','fieldName'] \
                            else f"{k}: {v}"
            f.write(f"{rec},\n")
        f.write('},\n')

{'variableId': 1, 'name': 'Population - Older Adult', 'fieldName': 'percentover65', 'valueType': 'percentage', 'yearsAvailable': '[2018,2019,2020,2021,2022,2023]', 'disabled': 'false'}
{'variableId': 2, 'name': 'Population - Youth', 'fieldName': 'percentunder18', 'valueType': 'percentage', 'yearsAvailable': '[2018,2019,2020,2021,2022,2023]', 'disabled': 'false'}
{'variableId': 3, 'name': 'Age or Residents', 'fieldName': 'null', 'valueType': 'numeric', 'yearsAvailable': '[]', 'disabled': 'true'}
{'variableId': 4, 'name': 'Area', 'fieldName': 'null', 'valueType': 'numeric', 'yearsAvailable': '[]', 'disabled': 'true'}
{'variableId': 5, 'name': 'Population Density', 'fieldName': 'null', 'valueType': 'numeric', 'yearsAvailable': '[]', 'disabled': 'true'}
{'variableId': 6, 'name': 'Population - Veterans', 'fieldName': 'percentveteran', 'valueType': 'percentage', 'yearsAvailable': '[2018,2019,2020,2021,2022,2023]', 'disabled': 'false'}
{'variableId': 7, 'name': 'Race/Ethnicity - All Other Rac

In [6]:
var_df.group_by('category').agg(pl.col('variableId')).sort(by='category').to_pandas()

,category,variableId
0,Character,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
1,Economy,"[13, 14, 15, 16, 17, 18, 19, 20, 21]"
2,Education,"[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]"
3,Engagement,"[34, 35, 36, 37]"
4,Environment,"[38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]"
5,Health,"[49, 50, 51, 52, 53, 54, 55, 56, 57]"
6,Housing,"[58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 6..."
7,Safety,"[74, 75, 76, 77, 78, 79]"
8,Transportation,"[80, 81, 82, 83, 84, 85, 86, 87, 88]"
